In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from catboost import CatBoostClassifier


In [ ]:
train = pd.read_csv('/kaggle/input/spooky-author-identification/train.zip', usecols=['text', 'author'])
test = pd.read_csv('/kaggle/input/spooky-author-identification/test.zip')


In [ ]:
train.sample(10)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns='author'), train['author'], test_size=0.25, stratify=train['author'])


In [ ]:
X_train


In [ ]:
model = CatBoostClassifier(text_features=['text'], random_state=1234, auto_class_weights='Balanced', loss_function='MultiClass',
                           task_type='GPU', devices="0:1")


In [ ]:
model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=25)


In [ ]:
print(model.predict_proba(X_test)[0:5])
print(y_test[0:5])


In [ ]:
predictions = model.predict(X_test)


In [ ]:
predictions = [item for sublist in predictions for item in sublist]


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))


In [ ]:
model.fit(train.drop(columns='author'), train['author'], early_stopping_rounds=25)


In [ ]:
preds = model.predict_proba(test.drop(columns='id'))
preds_df = pd.DataFrame(preds, columns=['EAP','HPL','MWS'])
preds_df = pd.concat(objs=[test['id'], preds_df], axis="columns")


In [ ]:
preds_df.sample(5)


In [ ]:
preds_df.to_csv("submission.csv", index=False)


In [ ]:
pd.read_csv('/kaggle/input/spooky-author-identification/sample_submission.zip').head()
